In [ ]:
#| default_exp callbacks

# Callbacks

> Callback used in handlers

In [ ]:
#| export
import copy
import fastcore.all as fc
from operator import attrgetter
from cftime import date2num
import numpy as np
import pandas as pd
from marisco.configs import cfg, cdl_cfg

In [ ]:
#| hide
from marisco.configs import cdl_cfg
from marisco.utils import CompareDfsAndTfmCB
from marisco.utils import test_dfs

## Core

In [ ]:
#| export
class Callback(): order = 0

In [ ]:
#| exports
def run_cbs(cbs, obj=None):
    for cb in sorted(cbs, key=attrgetter('order')):
        if cb.__doc__: obj.logs.append(cb.__doc__)
        cb(obj)

In [ ]:
#| exports
class Transformer():
    def __init__(self, dfs, cbs=None): 
        self.cbs = cbs
        #self.dfs = {k: v.copy() for k, v in dfs.items()}

        self.dfs = self._copy(dfs)
        
        self.logs = []

    def _copy(self, dfs):
        if isinstance(dfs, dict):
            return {k: v.copy() for k, v in dfs.items()}
        else:
            return dfs.copy()
        
    def callback(self):
        run_cbs(self.cbs, self)
        
    def unique(self, col_name):
        "Distinct values of a specific column present in all groups"
        columns = [df.get(col_name) for df in self.dfs.values() if df.get(col_name) is not None]
        values = np.concatenate(columns) if columns else []
        return np.unique(values)
        
    def __call__(self):
        if self.cbs: self.callback()
        return self.dfs

Example:

In [ ]:
dfs = {'biota': pd.DataFrame({'id': [0, 1, 2], 'species': [0, 2, 0], 'depth': [2, 3, 4]}),
       'seawater': pd.DataFrame({'id': [0, 1, 2], 'depth': [3, 4, 5]})}
tfm = Transformer(dfs); tfm()
tfm.unique('species')

array([0, 2])

In [ ]:
tfm.unique('non_existing_var')

array([], dtype=float64)

## Geographical coordinates

In [ ]:
#| exports
class SanitizeLonLatCB(Callback):
    "Drop row when both longitude & latitude equal 0. Drop unrealistic longitude & latitude values. Convert longitude & latitude `,` separator to `.` separator."
    def __init__(self, verbose=False): fc.store_attr()
    def __call__(self, tfm):
        for grp, df in tfm.dfs.items():
            " Convert `,` separator to `.` separator"
            df['lon'] = [float(str(x).replace(',', '.')) for x in df['lon']]
            df['lat'] = [float(str(x).replace(',', '.')) for x in df['lat']]
            
            # mask zero values
            mask_zeroes = (df.lon == 0) & (df.lat == 0) 
            nZeroes = mask_zeroes.sum()
            if nZeroes and self.verbose: 
                print(f'The "{grp}" group contains {nZeroes} data points whose (lon, lat) = (0, 0)')
            
            # mask gps out of bounds, goob. 
            mask_goob = (df.lon < -180) | (df.lon > 180) | (df.lat < -90) | (df.lat > 90)
            nGoob = mask_goob.sum()
            if nGoob and self.verbose: 
                print(f'The "{grp}" group contains {nGoob} data points whose lon or lat are unrealistic. Outside -90 to 90 for latitude and -180 to 180 for longitude.')
                
            tfm.dfs[grp] = df.loc[~(mask_zeroes | mask_goob)]

In [ ]:
# Check that measurements located at (0,0) get removed
dfs = {'biota': pd.DataFrame({'lon': [0, 1, 0], 'lat': [0, 2, 0]})}
tfm = Transformer(dfs, cbs=[SanitizeLonLatCB()])
tfm()['biota']

expected = [1., 2.]
fc.test_eq(tfm()['biota'].iloc[0].to_list(), expected)

In [ ]:
# Check that comma decimal separator get replaced by point instead
dfs = {'biota': pd.DataFrame({'lon': ['45,2'], 'lat': ['43,1']})}
tfm = Transformer(dfs, cbs=[SanitizeLonLatCB()])
tfm()['biota']

expected = [45.2, 43.1]
fc.test_eq(tfm()['biota'].iloc[0].to_list(), expected)

In [ ]:
# Check that out of bounds lon or lat get removed
dfs = {'biota': pd.DataFrame({'lon': [-190, 190, 1, 2, 1.1], 'lat': [1, 2, 91, -91, 2.2]})}
tfm = Transformer(dfs, cbs=[SanitizeLonLatCB()])
tfm()['biota']

expected = [1.1, 2.2]
fc.test_eq(tfm()['biota'].iloc[0].to_list(), expected)

## Remapping

## Structural

Many data providers use a long format (e.g `lat, lon, radionuclide, value, unc, ...`) to store their data. When encoding as `netCDF`, it is often required to use a wide format (e.g `lat, lon, nuclide1_value, nuclide1_unc, nuclide2_value, nuclide2_unc, ...`). The class `ReshapeLongToWide` is designed to perform this transformation.

In [ ]:
#| exports
class ReshapeLongToWide(Callback):
    def __init__(self, columns=['nuclide'], values=['value'], 
                 num_fill_value=-999, str_fill_value='STR FILL VALUE'):
        "Convert data from long to wide with renamed columns."
        fc.store_attr()
        self.derived_cols = self._get_derived_cols()
    
    def _get_derived_cols(self):
        "Retrieve all possible derived vars (e.g 'unc', 'dl', ...) from configs."
        return [value['name'] for value in cdl_cfg()['vars']['suffixes'].values()]

    def renamed_cols(self, cols):
        "Flatten columns name."
        return [inner if outer == "value" else f'{inner}{outer}' if inner else outer
                for outer, inner in cols]

    def _get_unique_fill_value(self, df, idx):
        "Get a unique fill value for NaN replacement."
        fill_value = self.num_fill_value
        while (df[idx] == fill_value).any().any():
            fill_value -= 1
        return fill_value

    def _fill_nan_values(self, df, idx):
        "Fill NaN values in index columns."
        num_fill_value = self._get_unique_fill_value(df, idx)
        for col in idx:
            fill_value = num_fill_value if pd.api.types.is_numeric_dtype(df[col]) else self.str_fill_value
            df[col] = df[col].fillna(fill_value)
        return df, num_fill_value

    def pivot(self, df):
        derived_coi = [col for col in self.derived_cols if col in df.columns]
        df.index.name = 'org_index'
        df = df.reset_index()
        idx = list(set(df.columns) - set(self.columns + derived_coi + self.values))
        
        df, num_fill_value = self._fill_nan_values(df, idx)

        pivot_df = df.pivot_table(index=idx,
                                  columns=self.columns,
                                  values=self.values + derived_coi,
                                  fill_value=np.nan,
                                  aggfunc=lambda x: x).reset_index()

        pivot_df[idx] = pivot_df[idx].replace({self.str_fill_value: np.nan, num_fill_value: np.nan})
        return pivot_df.set_index('org_index')

    def __call__(self, tfm):
        for grp in tfm.dfs.keys():
            tfm.dfs[grp] = self.pivot(tfm.dfs[grp])
            tfm.dfs[grp].columns = self.renamed_cols(tfm.dfs[grp].columns)

In [ ]:
dfs_test = fc.load_pickle('./files/pkl/dfs_reshape_test_in.pkl')
dfs_expected = fc.load_pickle('./files/pkl/dfs_reshape_test_expected.pkl')

tfm = Transformer(dfs_test, cbs=[ReshapeLongToWide()])
dfs_output = tfm()
test_dfs(dfs_output, dfs_expected)    

## Time

In [ ]:
#| exports
class EncodeTimeCB(Callback):
    "Encode time as `int` representing seconds since xxx"    
    def __init__(self, cfg , verbose=False): fc.store_attr()
    def __call__(self, tfm): 
        def format_time(x): 
            return date2num(x, units=self.cfg['units']['time'])
        
        for k in tfm.dfs.keys():
            # If invalid time entries.
            if tfm.dfs[k]['time'].isna().any():
                if self.verbose:
                    invalid_time_df=tfm.dfs[k][tfm.dfs[k]['time'].isna()]
                    print (f'{len(invalid_time_df.index)} of {len(tfm.dfs[k].index)} entries for `time` are invalid for {k}.')
                # Filter nan values
                tfm.dfs[k] = tfm.dfs[k][tfm.dfs[k]['time'].notna()]
            
            tfm.dfs[k]['time'] = tfm.dfs[k]['time'].apply(format_time)